# Supply Chain Management

## Output Data 1
**Create a resultant CSV/Excel file with the following columns:**
-  Order ID
-  AWB Number
-  Total weight as per X (KG)
-  Weight slab as per X (KG)
-  Total weight as per Courier Company (KG)
-  Weight slab charged by Courier Company (KG)
-  Delivery Zone as per X
-  Delivery Zone charged by Courier Company
-  Expected Charge as per X (Rs.)
-  Charges Billed by Courier Company (Rs.)
-  Difference Between Expected Charges and Billed Charges (Rs.)

In [49]:
# importing libraries
import pandas as pd 
import numpy as np

In [50]:
## loading data

order_report = pd.read_excel("Company X - Order Report.xlsx") 
pincode_zones = pd.read_excel("Company X - Pincode Zones.xlsx")
sku_master = pd.read_excel("Company X - SKU Master.xlsx")
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
rates = pd.read_excel("Courier Company - Rates.xlsx") 

In [51]:
# rename ExternOrderNo to Order ID to make easier to merge data
order_report.rename(columns = {"ExternOrderNo": "Order ID"}, inplace = True)   # Column ka naam change kiya

In [55]:
# showing the data of order_report
order_report

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [57]:
# Add new column : Total weight per X (KG) 
sku_master["Total Weight X (kg)"] = sku_master["Weight (g)"]/1000         # Gram ko KG me convert krna
print(sku_master)
sku_master.drop("Weight (g)", axis = 1, inplace = True)

              SKU  Weight (g)  Total Weight X (kg)
0   8904223815682         210                0.210
1   8904223815859         165                0.165
2   8904223815866         113                0.113
3   8904223815873          65                0.065
4   8904223816214         120                0.120
..            ...         ...                  ...
61  8904223819505         210                0.210
62  8904223819499         210                0.210
63  8904223819512         210                0.210
64  8904223819543         300                0.300
65     SACHETS001          10                0.010

[66 rows x 3 columns]


In [58]:
# rename Zone to Zone Charged Courier to make easier to merge data

invoice.rename(columns = {"Zone" : "Zone Charged Courier"}, inplace = True)

In [59]:
# rename Charged Weight to Total Weight Courier(kg) to make easier to merge data

invoice.rename(columns = {"Charged Weight" : "Total Weight Courier(kg)"}, inplace = True)

In [62]:
# Creating the Weight slab for X company

sku_master["Weight slab as per X (kg)" ] = np.where(sku_master["Total Weight X (kg)"] < 0.501, 0.5,
                                               np.where(sku_master["Total Weight X (kg)"] < 1.01 , 1,
                                                        sku_master["Total Weight X (kg)"]))

In [63]:
# Checking the weight slab columns

sku_master.head()

,SKU,Total Weight X (kg),Weight slab as per X (kg)
0,8904223815682,0.210,0.5
1,8904223815859,0.165,0.5
2,8904223815866,0.113,0.5
3,8904223815873,0.065,0.5
4,8904223816214,0.120,0.5


In [65]:
# Total Weight Courier(kg) to convert Total Weight slab Courier (kg)

invoice["Total Weight slab Courier (kg)"] = np.where(invoice["Total Weight Courier(kg)"] < 0.51, 0.5,
                                                     np.where(invoice["Total Weight Courier(kg)"] < 1.01, 1,
                                                     np.where(invoice["Total Weight Courier(kg)"] < 1.51, 1.5,
                                                     np.where(invoice["Total Weight Courier(kg)"] < 2.01, 2,
                                                     np.where(invoice["Total Weight Courier(kg)"] < 2.51, 2.5,
                                                     np.where(invoice["Total Weight Courier(kg)"] < 3.01, 3,
                                                             (invoice["Total Weight Courier(kg)"])))))))

In [107]:
# top 5 data and Checking the weight slab courier
invoice.head()

,AWB Code,Order ID,Total Weight Courier(kg),Warehouse Pincode,Customer Pincode,Zone Charged Courier,Type of Shipment,Billing Amount (Rs.),Total Weight slab Courier (kg)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5


In [67]:
# delivery zone as per X
print(pincode_zones.columns)

Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')


In [68]:
# rename Zone to Zone Z to make easier to merge data
pincode_zones.rename(columns = {"Zone": "Zone X"}, inplace = True)

In [69]:
# Showing data of pincode_zones
pincode_zones

,Warehouse Pincode,Customer Pincode,Zone X
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [118]:
# merging the data which have common column(order_report and sku_master)

X1 = order_report.merge(sku_master, how = "inner")   

In [119]:

# merging the data which have common column(pincode_zones and invoice)

X2 = pincode_zones.merge(invoice, how = "inner")

In [120]:
# merging the X1 and X2

merge_data = X1.merge(X2, how = "inner")

In [121]:
# Showing the data after final merging data
merge_data

,Order ID,SKU,Order Qty,Total Weight X (kg),Weight slab as per X (kg),Warehouse Pincode,Customer Pincode,Zone X,AWB Code,Total Weight Courier(kg),Zone Charged Courier,Type of Shipment,Billing Amount (Rs.),Total Weight slab Courier (kg)
0,2001827036,8904223818706,1.0,0.127,0.5,121003,173213,e,1091122418320,1.60,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,0.150,0.5,121003,173213,e,1091122418320,1.60,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,0.100,0.5,121003,173213,e,1091122418320,1.60,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,0.165,0.5,121003,173213,e,1091122418320,1.60,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,0.350,0.5,121003,173213,e,1091122418320,1.60,b,Forward charges,117.9,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,2001806229,8904223818942,1.0,0.133,0.5,121003,421204,d,1091117222080,0.71,d,Forward charges,90.2,1.0
563,2001806229,8904223818850,1.0,0.240,0.5,121003,421204,d,1091117222080,0.71,d,Forward charges,90.2,1.0
564,2001806226,8904223818850,2.0,0.240,0.5,121003,723146,d,1091117222065,0.68,d,Forward charges,90.2,1.0
565,2001806210,8904223816214,1.0,0.120,0.5,121003,140604,b,1091117221940,2.92,b,Forward charges,174.5,3.0


In [75]:
# Calculate the expected charge as per x company

charges = {
    "fwd_a_fixed": 29.5, "fwd_a_additional": 23.6,	# These are the fixed rates by courier company for forwad charges
    "fwd_b_fixed": 33, "fwd_b_additional": 28.3,	
    "fwd_c_fixed": 40.1, "fwd_c_additional": 38.9,	
    "fwd_d_fixed": 45.4, "fwd_d_additional": 44.8,	
    "fwd_e_fixed": 56.6, "fwd_e_additional": 55.5,	
    "rto_a_fixed": 13.6, "rto_a_additional": 23.6,	# These are the fixed rates by courier company for rto charges 
    "rto_b_fixed": 20.5, "rto_b_additional": 28.3,	
    "rto_c_fixed": 31.9, "rto_c_additional": 38.9,	
    "rto_d_fixed": 41.3, "rto_d_additional": 44.8,	
    "rto_e_fixed": 50.7, "rto_e_additional": 55.5
}
fixed_slab = 0.5 # this is a fixed weight slab where a fixed rate is applied on the first 0.5 of the weight slab and an additional rate is applied on the rest
expe_char_X = []

for i,j,k in zip(merge_data["Zone X"], merge_data["Weight slab as per X (kg)"], merge_data["Type of Shipment"]):
    a = j - fixed_slab # 
    if  i == "a":
        fix_price = charges["fwd_a_fixed"] + ((a/0.5)*charges["fwd_a_additional"]) # formula of calculate forwad charges

        if k == "Forward and RTO charges":
            fix_price = (fix_price + charges["rto_a_fixed"] + ((a/0.5)*charges["rto_a_additional"])) # formula of calculate rto charges
        expe_char_X.append(fix_price)        # append the charges amount

    elif i == "b":
        fix_price = charges["fwd_b_fixed"] + ((a/0.5)*charges["fwd_b_additional"])
        
        if k == "Forward and RTO charges":
            fix_price = (fix_price + charges["rto_b_fixed"] + ((a/0.5)* charges["rto_b_additional"]))
        expe_char_X.append(fix_price)

    elif i == "c":
        fix_price = charges["fwd_c_fixed"] + ((a/0.5)*charges["fwd_c_additional"])

        if k == "Forward and RTO charges":
            fix_price = (fix_price + charges["rto_c_fixed"] + ((a/0.5)* charges["rto_c_additional"]))
        expe_char_X.append(fix_price)                                  

    elif i == "d":
        fix_price = charges["fwd_d_fixed"] + ((a/0.5)*charges["fwd_d_additional"])

        if k == "Forward and RTO charges":
            fix_price = (fix_price + charges["rto_d_fixed"] + ((a/0.5)* charges["rto_d_additional"]))
        expe_char_X.append(fix_price)

    elif i == "e":
        fix_price = charges["fwd_e_fixed"] + ((a/0.5)*charges["fwd_e_additional"])

        if k == "Forward and RTO charges":
            fix_price = fix_price + charges["rto_e_fixed"] + ((a/0.5)* charges["rto_e_additional"])
        expe_char_X.append(fix_price)

In [125]:
# making a new column in merge_data and fill the all values

merge_data["Expected Charge as per X (Rs.)"] = expe_char_X

In [126]:
# making a new column in merge data and calculate Difference between expected charges as per x and billed charges as courier company

merge_data["Difference Between Expected Charges and Billed Charges (Rs.)"] = merge_data["Billing Amount (Rs.)"] - merge_data["Expected Charge as per X (Rs.)"]

In [123]:
# Droping the columns

merge_data.drop("Order Qty", axis = 1, inplace = True)
merge_data.drop("Warehouse Pincode" , axis = 1, inplace = True)
merge_data.drop("Customer Pincode", axis = 1, inplace = True)
merge_data.drop("Type of Shipment",axis = 1, inplace = True)
merge_data.drop("SKU", axis = 1, inplace = True)

In [127]:
# Final data(Output 1)

merge_data

,Order ID,Total Weight X (kg),Weight slab as per X (kg),Zone X,AWB Code,Total Weight Courier(kg),Zone Charged Courier,Billing Amount (Rs.),Total Weight slab Courier (kg),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,0.127,0.5,e,1091122418320,1.60,b,117.9,2.0,56.6,61.3
1,2001827036,0.150,0.5,e,1091122418320,1.60,b,117.9,2.0,56.6,61.3
2,2001827036,0.100,0.5,e,1091122418320,1.60,b,117.9,2.0,56.6,61.3
3,2001827036,0.165,0.5,e,1091122418320,1.60,b,117.9,2.0,56.6,61.3
4,2001827036,0.350,0.5,e,1091122418320,1.60,b,117.9,2.0,56.6,61.3
...,...,...,...,...,...,...,...,...,...,...,...
562,2001806229,0.133,0.5,d,1091117222080,0.71,d,90.2,1.0,45.4,44.8
563,2001806229,0.240,0.5,d,1091117222080,0.71,d,90.2,1.0,45.4,44.8
564,2001806226,0.240,0.5,d,1091117222065,0.68,d,90.2,1.0,45.4,44.8
565,2001806210,0.120,0.5,b,1091117221940,2.92,b,174.5,3.0,33.0,141.5


## Output Data 2
**Create a summary table**

1. Total orders where X has been correctly charged
 - Count, total invoice amount
2. Total Orders where X has been overcharged
  - Count, overcharging amount
3. Total Orders where X has been undercharged
  - Count, total undercharging amount

In [99]:
# Total orders where X has been correctly charged

correctly_charge = (merge_data["Billing Amount (Rs.)"] == merge_data["Expected Charge as per X (Rs.)"]).sum()
correctly_charge

15

In [100]:
# Total amount of orders where X has been correctly charged

a =  (merge_data["Billing Amount (Rs.)"] == merge_data["Expected Charge as per X (Rs.)"])
total_invoice_amount = merge_data[a]["Expected Charge as per X (Rs.)"].sum()
total_invoice_amount

606.5999999999999

In [101]:
# Total Orders where X has been overcharged

overcharged = (merge_data["Expected Charge as per X (Rs.)"] < merge_data["Billing Amount (Rs.)"]).sum()

overcharged

539

In [102]:
# Total amount of Orders where X has been overcharged

b = merge_data["Expected Charge as per X (Rs.)"] < merge_data["Billing Amount (Rs.)"]

total_overcharging_amount = merge_data[b]["Billing Amount (Rs.)"].sum()

total_overcharging_amount

67068.1

In [103]:
# Total Orders where X has been undercharged

undercharged = (merge_data["Expected Charge as per X (Rs.)"] > merge_data["Billing Amount (Rs.)"]).sum()

undercharged

13

In [104]:
# Total amount of Orders where X has been undercharged

c = merge_data["Expected Charge as per X (Rs.)"] > merge_data["Billing Amount (Rs.)"]

total_undercharging_amount = merge_data[c]["Expected Charge as per X (Rs.)"].sum()

total_undercharging_amount

837.2

In [106]:
table = {
    "correctly_charge" :[correctly_charge, total_invoice_amount],
    "overcharged" :[overcharged, total_overcharging_amount],
    "undercharged" :[undercharged, total_undercharging_amount]
}

index = ["Count", "Total_amount"]

summary_table = pd.DataFrame(table, index = index) # Convert dic into DataFrame
summary_table

,correctly_charge,overcharged,undercharged
Count,15.0,539.0,13.0
Total_amount,606.6,67068.1,837.2


In [130]:
# Convert DataFrame to CSV file
merge_data.to_excel("Output_1.xlsx", index= False)
summary_table.to_excel("output_2.xlsx", index = False)